<a href="https://colab.research.google.com/github/brucker3/KDM_spring2021/blob/main/ICP/ICP-5/ICP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# intsall spark and importing necessry packages 
!pip install pyspark
!pip install nltk
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Word2Vec
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet')

# creating spark session
spark = SparkSession.builder.appName("ICP5").getOrCreate()


     |████████████████████████████████| 204.8MB 66kB/s 
     |████████████████████████████████| 204kB 39.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=6cf3612c8a42828524bb5ffb45aaf9ccad7ae082d7492721bc469d4b0a02da67
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:

# read in of all 5 txt files 
# I tried to do this a cleaner way but spark kept skipping over some of the files 

txt1 = "../content/txt1.txt"
txt2 = "../content/txt2.txt"
txt3 = "../content/txt3.txt"
txt4 = "../content/txt4.txt"
txt5 = "../content/txt5.txt"

# opening and converting read in files to use able form 
txt1 = open(txt1, "r")
txt2 = open(txt2, "r")
txt3 = open(txt3, "r")
txt4 = open(txt4, "r")
txt5 = open(txt5, "r")
text1_sring = txt1.read()
text2_sring = txt2.read()
text3_sring = txt3.read()
text4_sring = txt4.read()
text5_sring = txt5.read()
# creating spark dataframe wiht the input data. 
sentenceData = spark.createDataFrame([
        (0, text1_sring),
        (1, text2_sring),
        (2, text3_sring),
        (3, text4_sring),
        (4, text5_sring)
    ], ["label", "sentence"])


word_to_vec = spark.createDataFrame([
        ( text1_sring.split(" "),),
        ( text2_sring.split(" "),),
        ( text3_sring.split(" "),),
        ( text4_sring.split(" "),),
        ( text5_sring.split(" "),)
    ], ["text"])



In [3]:
# tokenizer to set up for IDF calculation 
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|    0|When you ask Amer...|[when, you, ask, ...|
|    1|So a new coach is...|[so, a, new, coac...|
|    2|Whether it's in t...|[whether, it's, i...|
|    3|This is not every...|[this, is, not, e...|
|    4|But in 2002, when...|[but, in, 2002,, ...|
+-----+--------------------+--------------------+



In [4]:

def TF(words):
    # applying tf on the words data
    # alternatively, CountVectorizer can also be used to get term frequency vectors
    hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10)
    featurizedData = hashingTF.transform(words)
    featurizedData.cache()
    # calculating the IDF
    idf = IDF(inputCol="rawFeatures", outputCol="features")
    idfModel = idf.fit(featurizedData)
    rescaledData = idfModel.transform(featurizedData)
    return rescaledData

# for the orignal input 
#type(wordsData)
wordsData.show()
ouput = TF(wordsData)
ouput.show()  

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|    0|When you ask Amer...|[when, you, ask, ...|
|    1|So a new coach is...|[so, a, new, coac...|
|    2|Whether it's in t...|[whether, it's, i...|
|    3|This is not every...|[this, is, not, e...|
|    4|But in 2002, when...|[but, in, 2002,, ...|
+-----+--------------------+--------------------+

+-----+--------------------+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawFeatures|            features|
+-----+--------------------+--------------------+--------------------+--------------------+
|    0|When you ask Amer...|[when, you, ask, ...|(10,[0,1,2,3,4,5,...|(10,[0,1,2,3,4,5,...|
|    1|So a new coach is...|[so, a, new, coac...|(10,[0,1,2,3,4,5,...|(10,[0,1,2,3,4,5,...|
|    2|Whether it's in t...|[whether, it's, i...|(10,[0,1,2,3,4,5,...|(10,[0,1,2,3,4,5,..

In [5]:
#for lemmitized inputs 
# itterate thru the inputs and lemitize the words 
lemmatizer = WordNetLemmatizer()
wordset = wordsData.select('words').collect() 
for i in range(0,len(wordset)):
  for j in range(0,len(wordset[i][0])):
    wordset[i][0][j] = lemmatizer.lemmatize(wordset[i][0][j])
    
# recast list into a data frame and pass it to the TF-IDF functions 
wordset = spark.createDataFrame(wordset)
output_2 = TF(wordset)
output_2.select("words", "features").show(10, truncate = True)


+--------------------+--------------------+
|               words|            features|
+--------------------+--------------------+
|[when, you, ask, ...|(10,[0,1,2,3,4,5,...|
|[so, a, new, coac...|(10,[0,1,2,3,4,5,...|
|[whether, it's, i...|(10,[0,1,2,3,4,5,...|
|[this, is, not, e...|(10,[0,1,2,3,4,5,...|
|[but, in, 2002,, ...|(10,[0,1,2,3,4,5,...|
+--------------------+--------------------+



In [7]:
#creating NGrams 
# output looks good for the set up 
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")
ngramDataFrame = ngram.transform(wordsData)
ngramDataFrame.show()
output_3 = TF(ngramDataFrame)
output_3.show()

+-----+--------------------+--------------------+--------------------+
|label|            sentence|               words|              ngrams|
+-----+--------------------+--------------------+--------------------+
|    0|When you ask Amer...|[when, you, ask, ...|[when you, you as...|
|    1|So a new coach is...|[so, a, new, coac...|[so a, a new, new...|
|    2|Whether it's in t...|[whether, it's, i...|[whether it's, it...|
|    3|This is not every...|[this, is, not, e...|[this is, is not,...|
|    4|But in 2002, when...|[but, in, 2002,, ...|[but in, in 2002,...|
+-----+--------------------+--------------------+--------------------+

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|            sentence|               words|              ngrams|         rawFeatures|            features|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|    0|When you ask A

In [ ]:
#2 word to vec 
# data set now set up with word to vector 
word2Vec = Word2Vec(vectorSize=2, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(word_to_vec)
result = model.transform(word_to_vec)
print(result.show())
# change column name so it can be passed to TF function 
result = reslut.withColumnRenamed('text', 'words')
ouput_nonlp = TF(result)
ouput_nonlp.show()



+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|[When, you, ask, ...|[-0.0251492020776...|
|[So, a, new, coac...|[0.01094523886914...|
|[Whether, it's, i...|[0.01033639257945...|
|[This, is, not, e...|[0.00965418570558...|
|[But, in, 2002,, ...|[-0.0256646900093...|
+--------------------+--------------------+

None
+--------------------+--------------------+--------------------+--------------------+
|               words|              result|         rawFeatures|            features|
+--------------------+--------------------+--------------------+--------------------+
|[When, you, ask, ...|[-0.0251492020776...|(10,[0,1,2,3,4,5,...|(10,[0,1,2,3,4,5,...|
|[So, a, new, coac...|[0.01094523886914...|(10,[0,1,2,3,4,5,...|(10,[0,1,2,3,4,5,...|
|[Whether, it's, i...|[0.01033639257945...|(10,[0,1,2,3,4,5,...|(10,[0,1,2,3,4,5,...|
|[This, is, not, e...|[0.00965418570558...|(10,[0,1,2,3,4,5,...|(10,[0,1,2,3,4,5,.

In [ ]:

# lemitization for quesiton 2 part b 
# wordset two can now be used for quesiton 2 part b 
wordset2 = word_to_vec.select('text').collect() 
for i in range(0,len(wordset2)):
    for j in range(0,len(wordset2[i][0])):
        wordset2[i][0][j] = lemmatizer.lemmatize(wordset2[i][0][j])

wordset2 = spark.createDataFrame(wordset2)
word2Vec = Word2Vec(vectorSize=2, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(wordset2)
result = model.transform(wordset2)
print(result.show(10))


+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|[When, you, ask, ...|[-0.0271476942115...|
|[So, a, new, coac...|[0.00787496337234...|
|[Whether, it's, i...|[0.00708858389407...|
|[This, is, not, e...|[-0.0048036088046...|
|[But, in, 2002,, ...|[-0.0131841920429...|
+--------------------+--------------------+

None


In [ ]:
# word vec with ngram 
#word2Vec_ngram = spark.createDataFrame(word_to_vec)
# toke the orignal ngrams data set 
# then re ran the w2vec conde on it 
ngram = spark.createDataFrame(ngramDataFrame.select("ngrams").collect())
word2Vec = Word2Vec(vectorSize=2, minCount=0, inputCol="ngrams", outputCol="result")
model = word2Vec.fit(ngramDataFrame)
result = model.transform(ngramDataFrame)
print(result.show())
output_3 = TF(result)
output_3.show()



+-----+--------------------+--------------------+--------------------+--------------------+
|label|            sentence|               words|              ngrams|              result|
+-----+--------------------+--------------------+--------------------+--------------------+
|    0|When you ask Amer...|[when, you, ask, ...|[when you, you as...|[-0.0133773904573...|
|    1|So a new coach is...|[so, a, new, coac...|[so a, a new, new...|[0.00359556692186...|
|    2|Whether it's in t...|[whether, it's, i...|[whether it's, it...|[-0.0276695854587...|
|    3|This is not every...|[this, is, not, e...|[this is, is not,...|[-0.0131260473988...|
|    4|But in 2002, when...|[but, in, 2002,, ...|[but in, in 2002,...|[0.01039087617495...|
+-----+--------------------+--------------------+--------------------+--------------------+

None
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|label|            sentence|    